In [1]:
from textblob import TextBlob
from dataclasses import dataclass
from dotenv import load_dotenv
import pip._vendor.requests as requests # explicitly importing because its ipynb in vscode
import os
import pandas as pd

In [2]:
# Grab your access token for your account
def configure():
    load_dotenv()

auth = requests.auth.HTTPBasicAuth(os.getenv('CLIENT_ID'), os.getenv('SECRET_KEY'))
data = {
    'grant_type': 'password',
    'username': os.getenv('USERNAME'),
    'password': os.getenv('USER_PASSWORD')
}
headers = {'User-Agent': 'MyAPI/0.0.1'}
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']


In [3]:
# Pass the new headers with the authoization token to make our request
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}
#requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()
def get_subreddit(subreddit: str, sort: str) -> requests:
    
    reqString = 'https://oauth.reddit.com/r/' + subreddit + '/' + sort
    retReq = requests.get(reqString, headers=headers, params={'limit': 100})
    return retReq


In [4]:
# Example of using 'after'
# See how to grab the data back in time below 
#hot = requests.get('https://oauth.reddit.com/r/leagueoflegends/hot', headers=headers, params={'limit': '100', 'after': 't3_1cj4wff'})

In [5]:
def parse_subreddit(req: requests) -> pd.DataFrame:
    index = 0
    new_df = pd.DataFrame()
    for post in req.json()['data']['children']:
        df_new_row = pd.DataFrame({
            'subreddit' : post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'upvotes': post['data']['ups'],
            'downvotes': post['data']['downs'],
            'score': post['data']['score']
            }, index=[index])
        new_df = pd.concat([new_df, df_new_row])
        index += 1
    return new_df

In [6]:
# How to extrapolate data back in time

# This returns the id of the last post (oldest)
# post['kind'] + '_' + post['data']['id']
# Take output from this and plug it into params using 'after': output(the ID)
# This will pull another 100 before 
# # To see a list of things you can do with the api
# post['data'].keys()


In [7]:

@dataclass 
class Mood:
    emoji: str
    sentiment: float

def average_mood(moods: Mood, threshold: float) -> Mood: 
    totalSentiment = []
    friendly_threshold: float = threshold
    hostile_threshold: float = -threshold

    for mood in moods:
        totalSentiment.append(mood.sentiment)

    calculatedSentimate = sum(totalSentiment) / len(totalSentiment)
    
    if calculatedSentimate >= friendly_threshold:
        return Mood('Positive', calculatedSentimate)
    elif calculatedSentimate <= hostile_threshold:
        return Mood('Negative', calculatedSentimate)
    else:
        return Mood('Neutral😐', calculatedSentimate)


def get_mood(input_text: str, *, threshold: float) -> Mood:
    sentiment: float = TextBlob(input_text).sentiment.polarity

    friendly_threshold: float = threshold
    hostile_threshold: float = -threshold

    if sentiment >= friendly_threshold:
        return Mood('Positive😊', sentiment)
    elif sentiment <= hostile_threshold:
        return Mood('Negative😡', sentiment)
    else:
        return Mood('Neutral😐', sentiment)
    
def calc_accuracy(df: pd.DataFrame, threshold: float) -> Mood:
    titleMoods = []
    postMoods = []
    for index, row in df.iterrows() :
        # print(row['title'], row['selftext'])
        titleMood: Mood = get_mood(row['title'], threshold=threshold)  
        postMood: Mood = get_mood(row['selftext'], threshold=threshold)
        titleMoods.append(titleMood)
        postMoods.append(postMood)
    avg: Mood= average_mood(titleMoods, threshold)

    return avg


In [6]:


    
if __name__ == '__main__':
    # subreddit = get_subreddit('cscareerquestions', 'new')
    # df = parse_subreddit(subreddit)
    # output = calc_accuracy(df, 0.3)
    # print(output)
    print("Hello, welcome to my reddit sentiment bot.")
    print("Type exit at any timeto stop the program.")
    print("Please enter a subreddit")
    subreddit = input("Text: ")
    
    while text.lower() != 'exit':
        print("Please type one of the following sorting methods: best | hot | new | top | rising")
        text: str = input('method: ')

        if text.lower == 'best' or 'hot' or 'new' or 'top' or 'rising':
            print("Searching for subreddit: ", subreddit)
            print("Getting the last 100 of: ", text)
            subreddit = get_subreddit(subreddit, text)
        else:
            print("Invalid entry, please try again.")
        
        

        #print(f'{mood.emoji} ({mood.sentiment})')


Hello, welcome to my reddit sentiment bot.
Type exit at any timeto stop the program.
Please enter a subreddit
Please type one of the following sorting methods: best | hot | new | top | rising
rising
Please type one of the following sorting methods: best | hot | new | top | rising
rising
Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods: best | hot | new | top | rising

Please type one of the following sorting methods

KeyboardInterrupt: Interrupted by user